In [2]:
### Desarrollado por Leandro Coronel

In [3]:
import os
import shutil
import numpy as np

# Reiniciamos el entorno para asegurar limpieza
if os.path.exists('datalake'):
    shutil.rmtree('datalake')

# Creamos las tres zonas de modelado
for layer in ['datalake/bronze', 'datalake/silver', 'datalake/gold']:
    os.makedirs(layer, exist_ok=True)

print("✅ Infraestructura de Data Lake creada.")

✅ Infraestructura de Data Lake creada.


In [4]:
import requests
import json

base_url = "https://fakestoreapi.com"
endpoints = ['products', 'users', 'carts']

print("📥 Ingestando datos crudos...")

for e in endpoints:
    # Simulamos la extracción de la fuente
    raw_data = requests.get(f"{base_url}/{e}").json()
    
    # Persistencia en formato nativo (JSON)
    with open(f'datalake/bronze/{e}.json', 'w') as f:
        json.dump(raw_data, f)

print(f"   -> {len(endpoints)} entidades cargadas en Bronze.")

📥 Ingestando datos crudos...
   -> 3 entidades cargadas en Bronze.


In [5]:
import pandas as pd

print("⚙️ Aplicando reglas de calidad (Silver Layer)...")

# --- 1. PRODUCTOS ---
# Regla: El precio viene como string o int, forzamos a Float para consistencia financiera.
df_prod = pd.read_json('datalake/bronze/products.json')
df_prod['price'] = df_prod['price'].astype(float)
df_prod.to_parquet('datalake/silver/products.parquet')

# --- 2. USUARIOS ---
# Regla de Modelado: Aplanar la jerarquía.
# 'address' es un diccionario. Lo convertimos en columnas: address_city, address_street...
with open('datalake/bronze/users.json') as f:
    users_data = json.load(f)

# json_normalize es nuestra herramienta de normalización
df_users = pd.json_normalize(users_data, sep='_')
df_users.to_parquet('datalake/silver/users.parquet')

# --- 3. CARRITOS (Transacciones) ---
# Regla: Las fechas deben ser objetos manipulables, no texto.
df_carts = pd.read_json('datalake/bronze/carts.json')
df_carts['date'] = pd.to_datetime(df_carts['date'])
df_carts.to_parquet('datalake/silver/carts.parquet')

print("✅ Datos limpios y estandarizados en formato Parquet.")

⚙️ Aplicando reglas de calidad (Silver Layer)...
✅ Datos limpios y estandarizados en formato Parquet.


In [7]:
df_carts

,id,userId,date,products,__v
0,1,1,2020-03-02 00:00:00+00:00,"[{'productId': 1, 'quantity': 4}, {'productId'...",0
1,2,1,2020-01-02 00:00:00+00:00,"[{'productId': 2, 'quantity': 4}, {'productId'...",0
2,3,2,2020-03-01 00:00:00+00:00,"[{'productId': 1, 'quantity': 2}, {'productId'...",0
3,4,3,2020-01-01 00:00:00+00:00,"[{'productId': 1, 'quantity': 4}]",0
4,5,3,2020-03-01 00:00:00+00:00,"[{'productId': 7, 'quantity': 1}, {'productId'...",0
5,6,4,2020-03-01 00:00:00+00:00,"[{'productId': 10, 'quantity': 2}, {'productId...",0
6,7,8,2020-03-01 00:00:00+00:00,"[{'productId': 18, 'quantity': 1}]",0


In [11]:
# Leemos desde Silver (nuestra fuente limpia)
s_prod = pd.read_parquet('datalake/silver/products.parquet')
s_user = pd.read_parquet('datalake/silver/users.parquet')

# --- DIMENSIÓN PRODUCTO ---
# Selección de atributos maestros
dim_product = s_prod[['id', 'title', 'category', 'price']].copy()
dim_product.columns = ['product_id', 'product_name', 'category', 'unit_price']

# --- DIMENSIÓN USUARIO ---
# Selección de atributos demográficos y geográficos
dim_user = s_user[['id', 'username', 'email', 'address_city']].copy()
dim_user.columns = ['user_id', 'username', 'email', 'city']

# Materialización
dim_product.to_parquet('datalake/gold/dim_product.parquet')
dim_user.to_parquet('datalake/gold/dim_user.parquet')

# Definir rango con timezone UTC
start_date = pd.Timestamp("2020-01-01 00:00:00", tz="UTC")
end_date = pd.Timestamp("2030-12-31 00:00:00", tz="UTC")

# Crear rango diario
fechas = pd.date_range(start=start_date, end=end_date, freq="D")

# Crear DataFrame
dim_calendario = pd.DataFrame({"fecha": fechas})

# ===== Atributos de la dimensión =====

dim_calendario["fecha_key"] = dim_calendario["fecha"].dt.strftime("%Y%m%d").astype(int)

dim_calendario["anio"] = dim_calendario["fecha"].dt.year
dim_calendario["mes"] = dim_calendario["fecha"].dt.month
dim_calendario["nombre_mes"] = dim_calendario["fecha"].dt.month_name()
dim_calendario["mes_abrev"] = dim_calendario["fecha"].dt.strftime("%b")

dim_calendario["trimestre"] = dim_calendario["fecha"].dt.quarter
dim_calendario["semana_anio"] = dim_calendario["fecha"].dt.isocalendar().week.astype(int)

dim_calendario["dia"] = dim_calendario["fecha"].dt.day
dim_calendario["dia_anio"] = dim_calendario["fecha"].dt.dayofyear

dim_calendario["dia_semana"] = dim_calendario["fecha"].dt.weekday + 1  # 1=Lunes
dim_calendario["nombre_dia"] = dim_calendario["fecha"].dt.day_name()

dim_calendario["es_fin_semana"] = dim_calendario["fecha"].dt.weekday >= 5
dim_calendario["es_inicio_mes"] = dim_calendario["fecha"].dt.is_month_start
dim_calendario["es_fin_mes"] = dim_calendario["fecha"].dt.is_month_end
dim_calendario["es_inicio_anio"] = dim_calendario["fecha"].dt.is_year_start
dim_calendario["es_fin_anio"] = dim_calendario["fecha"].dt.is_year_end

# Ordenar columnas
dim_calendario = dim_calendario.sort_values("fecha").reset_index(drop=True)

dim_calendario.to_parquet('datalake/gold/dim_calendario.parquet')

print("✅ Dimensiones creadas.")

✅ Dimensiones creadas.


In [10]:
dim_calendario.head()

,fecha,fecha_key,anio,mes,nombre_mes,mes_abrev,trimestre,semana_anio,dia,dia_anio,dia_semana,nombre_dia,es_fin_semana,es_inicio_mes,es_fin_mes,es_inicio_anio,es_fin_anio
0,2020-01-01 00:00:00+00:00,20200101,2020,1,January,Jan,1,1,1,1,3,Wednesday,False,True,False,True,False
1,2020-01-02 00:00:00+00:00,20200102,2020,1,January,Jan,1,1,2,2,4,Thursday,False,False,False,False,False
2,2020-01-03 00:00:00+00:00,20200103,2020,1,January,Jan,1,1,3,3,5,Friday,False,False,False,False,False
3,2020-01-04 00:00:00+00:00,20200104,2020,1,January,Jan,1,1,4,4,6,Saturday,True,False,False,False,False
4,2020-01-05 00:00:00+00:00,20200105,2020,1,January,Jan,1,1,5,5,7,Sunday,True,False,False,False,False


In [12]:
s_cart = pd.read_parquet('datalake/silver/carts.parquet')

# 1. CAMBIO DE GRANULARIDAD (La clave del modelado)
# Convertimos la lista de productos en filas individuales
fact_sales = s_cart.explode('products').reset_index(drop=True)

# 2. EXTRACCIÓN DE LLAVES FORÁNEAS (FK)
# Ahora cada fila tiene un diccionario {'productId': 1, 'quantity': 2}
# Extraemos esos valores para que sean columnas relacionables
fact_sales['product_id'] = fact_sales['products'].apply(lambda x: x['productId'])
fact_sales['quantity'] = fact_sales['products'].apply(lambda x: x['quantity'])

# 3. ENRIQUECIMIENTO (Cálculo de Métricas)
# El hecho "Venta" necesita el monto ($). El carrito no trae precios.
# Hacemos un JOIN con la dimensión producto para traer el precio unitario.
fact_sales = fact_sales.merge(dim_product[['product_id', 'unit_price']], on='product_id', how='left')

# Métrica Calculada: Cantidad * Precio Unitario
fact_sales['total_amount'] = fact_sales['quantity'] * fact_sales['unit_price']

# 4. DIMENSIÓN TIEMPO DERIVADA
# Extraemos atributos de fecha para facilitar el filtrado
fact_sales['year'] = fact_sales['date'].dt.year
fact_sales['month'] = fact_sales['date'].dt.month
fact_sales['day'] = fact_sales['date'].dt.day

# 5. SELECCIÓN FINAL (Esquema Estrella)
# Nos quedamos solo con las llaves (FK) y las métricas numéricas
final_columns = [
    'id',           # Sale ID (Podemos usar el Cart ID como referencia)
    'userId',       # FK hacia Dim_User
    'product_id',   # FK hacia Dim_Product
    'date', 'year', 'month', # Dimensiones Temporales
    'quantity',     # Métrica 1
    'total_amount'  # Métrica 2
]

fact_sales_final = fact_sales[final_columns].rename(columns={'id': 'sale_id', 'userId': 'user_id'})

# Materialización
fact_sales_final.to_parquet('datalake/gold/fact_sales.parquet')

print("✅ Tabla de Hechos (Fact_Sales) creada con granularidad de línea de producto.")

✅ Tabla de Hechos (Fact_Sales) creada con granularidad de línea de producto.


In [13]:
fact_sales_final

,sale_id,user_id,product_id,date,year,month,quantity,total_amount
0,1,1,1,2020-03-02 00:00:00+00:00,2020,3,4,439.80
1,1,1,2,2020-03-02 00:00:00+00:00,2020,3,1,22.30
2,1,1,3,2020-03-02 00:00:00+00:00,2020,3,6,335.94
3,2,1,2,2020-01-02 00:00:00+00:00,2020,1,4,89.20
4,2,1,1,2020-01-02 00:00:00+00:00,2020,1,10,1099.50
5,2,1,5,2020-01-02 00:00:00+00:00,2020,1,2,1390.00
6,3,2,1,2020-03-01 00:00:00+00:00,2020,3,2,219.90
7,3,2,9,2020-03-01 00:00:00+00:00,2020,3,1,64.00
8,4,3,1,2020-01-01 00:00:00+00:00,2020,1,4,439.80
9,5,3,7,2020-03-01 00:00:00+00:00,2020,3,1,9.99


In [14]:
# Cargamos el modelo (simulando Power BI o Tableau)
gold_fact = pd.read_parquet('datalake/gold/fact_sales.parquet')
gold_prod = pd.read_parquet('datalake/gold/dim_product.parquet')

# Hacemos el JOIN entre Hechos y Dimensiones
modelo_estrella = gold_fact.merge(gold_prod, on='product_id')

# Agregación (GROUP BY)
reporte = modelo_estrella.groupby('category')[['quantity', 'total_amount']].sum().sort_values('total_amount', ascending=False)

# Formato visual
pd.options.display.float_format = '${:,.2f}'.format
print(reporte)

                  quantity  total_amount
category                                
men's clothing          31     $2,646.44
jewelery                 4     $1,410.98
electronics              6       $624.00
women's clothing         1         $9.85


In [15]:
modelo_estrella

,sale_id,user_id,product_id,date,year,month,quantity,total_amount,product_name,category,unit_price
0,1,1,1,2020-03-02 00:00:00+00:00,2020,3,4,$439.80,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,$109.95
1,1,1,2,2020-03-02 00:00:00+00:00,2020,3,1,$22.30,Mens Casual Premium Slim Fit T-Shirts,men's clothing,$22.30
2,1,1,3,2020-03-02 00:00:00+00:00,2020,3,6,$335.94,Mens Cotton Jacket,men's clothing,$55.99
3,2,1,2,2020-01-02 00:00:00+00:00,2020,1,4,$89.20,Mens Casual Premium Slim Fit T-Shirts,men's clothing,$22.30
4,2,1,1,2020-01-02 00:00:00+00:00,2020,1,10,"$1,099.50","Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,$109.95
5,2,1,5,2020-01-02 00:00:00+00:00,2020,1,2,"$1,390.00",John Hardy Women's Legends Naga Gold & Silver ...,jewelery,$695.00
6,3,2,1,2020-03-01 00:00:00+00:00,2020,3,2,$219.90,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,$109.95
7,3,2,9,2020-03-01 00:00:00+00:00,2020,3,1,$64.00,WD 2TB Elements Portable External Hard Drive -...,electronics,$64.00
8,4,3,1,2020-01-01 00:00:00+00:00,2020,1,4,$439.80,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,$109.95
9,5,3,7,2020-03-01 00:00:00+00:00,2020,3,1,$9.99,White Gold Plated Princess,jewelery,$9.99


In [29]:
fact_sales

,id,userId,date,products,__v,product_id,quantity,unit_price,total_amount,year,month,day
0,1,1,2020-03-02 00:00:00+00:00,"{'productId': 1, 'quantity': 4}",0,1,4,$109.95,$439.80,2020,3,2
1,1,1,2020-03-02 00:00:00+00:00,"{'productId': 2, 'quantity': 1}",0,2,1,$22.30,$22.30,2020,3,2
2,1,1,2020-03-02 00:00:00+00:00,"{'productId': 3, 'quantity': 6}",0,3,6,$55.99,$335.94,2020,3,2
3,2,1,2020-01-02 00:00:00+00:00,"{'productId': 2, 'quantity': 4}",0,2,4,$22.30,$89.20,2020,1,2
4,2,1,2020-01-02 00:00:00+00:00,"{'productId': 1, 'quantity': 10}",0,1,10,$109.95,"$1,099.50",2020,1,2
5,2,1,2020-01-02 00:00:00+00:00,"{'productId': 5, 'quantity': 2}",0,5,2,$695.00,"$1,390.00",2020,1,2
6,3,2,2020-03-01 00:00:00+00:00,"{'productId': 1, 'quantity': 2}",0,1,2,$109.95,$219.90,2020,3,1
7,3,2,2020-03-01 00:00:00+00:00,"{'productId': 9, 'quantity': 1}",0,9,1,$64.00,$64.00,2020,3,1
8,4,3,2020-01-01 00:00:00+00:00,"{'productId': 1, 'quantity': 4}",0,1,4,$109.95,$439.80,2020,1,1
9,5,3,2020-03-01 00:00:00+00:00,"{'productId': 7, 'quantity': 1}",0,7,1,$9.99,$9.99,2020,3,1


In [17]:
### 🟢 Nivel 1: Consultas Básicas (Agregaciones Simples)

In [28]:
#### 1. ¿Cuál es el Ingreso Total Histórico (Total Revenue) de la compañía?

#**Insight:** Conocer el tamaño del negocio.

#**Pista:** `fact_sales['total_amount'].sum()`

# Tu código aquí
total_revenue = fact_sales['total_amount'].sum()
print(f"💰 Ingreso Total: ${total_revenue:,.2f}")

💰 Ingreso Total: $4,691.27


In [30]:
#### 2. ¿Cuántas unidades (Quantity) se han vendido en total?

#**Insight:** Conocer el volumen de logística necesario.

#**Pista:** `fact_sales['quantity'].sum()`

# Tu código aquí
total_units = fact_sales['quantity'].sum()
print(f"📦 Unidades Vendidas: {total_units:,}")

📦 Unidades Vendidas: 42


In [25]:
#### 3. ¿Cuál es el promedio de precio de venta (Average Selling Price)?

#**Insight:** Entender el posicionamiento de precios de la tienda.

#**Pista:** `fact_sales['unit_price'].mean()` (Requiere cruce con Dim_Product si no persististe el precio en la Fact, o usar la métrica calculada).

# Tu código aquí
avg_price = fact_sales['unit_price'].mean()
print(f"💵 Precio Promedio: ${avg_price:,.2f}")

💵 Precio Promedio: $110.94


In [31]:
### 🟡 Nivel 2: Análisis Dimensional (Joins y Group By)

#**Objetivo:** Verificar que las Llaves Foráneas (FK) conectan correctamente con las Dimensiones.

In [32]:
#### 4. ¿Cuál es la Categoría de Productos más rentable?

#**Insight:** Identificar qué segmento del negocio "paga las cuentas".

#**Pista:**
#- Join `Fact_Sales` con `Dim_Product`
#- Agrupar por `category`
#- Sumar `total_amount`
#- Ordenar descendente


# Tu código aquí
categoria_rentable = modelo_estrella.groupby('category')['total_amount'].sum().sort_values(ascending=False)
print("📊 Ingresos por Categoría:")
print(categoria_rentable)

📊 Ingresos por Categoría:
category
men's clothing     $2,646.44
jewelery           $1,410.98
electronics          $624.00
women's clothing       $9.85
Name: total_amount, dtype: float64


In [33]:
#### 5. Top 5: ¿Cuáles son los productos "Estrella" (Más vendidos por ingresos)?

#**Insight:** Gestión de inventario y marketing para los productos líderes.

#**Pista:** Agrupar por `product_name` -> Sumar `total_amount` -> `.head(5)`


# Tu código aquí
top_productos = modelo_estrella.groupby('product_name')['total_amount'].sum().sort_values(ascending=False).head(5)
print("⭐ Top 5 Productos Estrella:")
print(top_productos)

⭐ Top 5 Productos Estrella:
product_name
Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops                         $2,199.00
John Hardy Women's Legends Naga Gold & Silver Dragon Station Chain Bracelet   $1,390.00
WD 4TB Gaming Drive Works with Playstation 4 Portable External Hard Drive       $342.00
Mens Cotton Jacket                                                              $335.94
SanDisk SSD PLUS 1TB Internal SSD - SATA III 6 Gb/s                             $218.00
Name: total_amount, dtype: float64


In [34]:
#### 6. ¿Cuál es la ciudad (City) con mayor volumen de compras?

#**Insight:** Identificar dónde concentrar los esfuerzos de logística o publicidad local.

#**Pista:** Join `Fact_Sales` con `Dim_User` -> Agrupar por `city`

# Tu código aquí
gold_user = pd.read_parquet('datalake/gold/dim_user.parquet')
ventas_por_ciudad = gold_fact.merge(gold_user, on='user_id').groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("🌆 Ventas por Ciudad:")
print(ventas_por_ciudad.head(10))

🌆 Ventas por Ciudad:
city
kilcoole      $3,660.64
San Antonio     $560.00
Cullman         $460.78
mesa              $9.85
Name: total_amount, dtype: float64


In [36]:
### 🔴 Nivel 3: Análisis Avanzado (Granularidad y Tiempo)

#**Objetivo:** Probar la lógica de negocio y el manejo de fechas.

In [37]:
#### 7. Análisis de Tendencia: ¿Cómo se comportan las ventas por mes?

#**Insight:** Detectar estacionalidad (ej. ¿se vende más en diciembre?).

#**Pista:** Usar la columna `month` o `date` de la Fact Table para agrupar y graficar una línea de tiempo.

# Tu código aquí
ventas_mensuales = fact_sales.groupby('month')['total_amount'].sum().sort_index()
print("📈 Tendencia de Ventas Mensuales:")
print(ventas_mensuales)

# Opcional: Graficar
# import matplotlib.pyplot as plt
# ventas_mensuales.plot(kind='line', marker='o')
# plt.title('Ventas por Mes')
# plt.xlabel('Mes')
# plt.ylabel('Ingresos ($)')
# plt.show()

📈 Tendencia de Ventas Mensuales:
month
1   $3,018.50
3   $1,672.77
Name: total_amount, dtype: float64
